<a href="https://colab.research.google.com/github/ovisaha5/DataProcessingSP22/blob/main/Quiz7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quiz7
##Avijit Saha

####Questions

1.   Maximum Duration From Closed and Active start station


2.   Average duration for the closed start station and the active start station.






In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
%%bigquery --project smart-tracer-338719
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 7

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
5,9900286143,24-Hour Kiosk (Austin B-cycle),924,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,17
6,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6


In [3]:
%%bigquery --project smart-tracer-338719
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
LIMIT 20

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,3464,Pease Park,closed,1155 Kingsbury St,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
1,2500,Republic Square,closed,425 W 4th Street,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
2,2536,Waller & 6th St.,closed,602 Waller St.,None,None,None,None,None,None,None,None,3,2021-01-04 12:00:00+00:00
3,2538,Bullock Museum @ Congress & MLK,closed,1881 Congress Ave.,None,None,None,None,None,None,None,None,1,2021-01-04 12:00:00+00:00
4,2541,State Capitol @ 14th & Colorado,closed,206 W. 14th St.,None,None,None,None,None,None,None,None,1,2021-01-04 12:00:00+00:00
5,2545,ACC - Rio Grande & 12th,closed,700 W. 12th St.,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
6,2546,ACC - West & 12th Street,closed,1231 West Ave.,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
7,2550,Republic Square @ Guadalupe & 4th St.,closed,Presented by Austin Ventures,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00
8,2564,5th & San Marcos,closed,991 East 5th,None,None,None,None,None,None,None,None,3,2021-01-04 12:00:00+00:00
9,2576,Rainey @ River St,closed,64 Rainey St,None,None,None,None,None,None,None,None,9,2021-01-04 12:00:00+00:00


In [10]:
%%bigquery --project smart-tracer-338719
SELECT name, status
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
WHERE status='active'



,name,status
0,OFFICE/Main/Shop/Repair,active
1,Boardwalk West,active
2,Congress & Cesar Chavez,active
3,Barton Springs Pool,active
4,Riverside @ S. Lamar,active
...,...,...
73,South Congress @ Bouldin Creek,active
74,MoPac Pedestrian Bridge @ Veterans Drive,active
75,Dean Keeton & Speedway,active
76,Dean Keeton & Whitis,active


In [13]:
%%bigquery --project smart-tracer-338719
SELECT COUNTIF(status='active')
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`


,f0_
0,78


###Which station has the maximum duration from closed and active start station?

First, I join both of the table

In [63]:
%%bigquery --project smart-tracer-338719
SELECT start_station_name, status, duration_minutes as duration
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`as station JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as trips ON station.name=trips.start_station_name
GROUP BY start_station_name, duration_minutes, status

,start_station_name,status,duration
0,Zilker Park West,closed,39
1,Toomey Rd @ South Lamar,closed,31
2,Toomey Rd @ South Lamar,closed,30
3,State Capitol @ 14th & Colorado,closed,19
4,State Capitol @ 14th & Colorado,closed,17
...,...,...,...
26454,Nueces & 26th,active,152
26455,Nueces & 26th,active,473
26456,Nueces & 26th,active,1012
26457,Nueces & 26th,active,652


Now, I will only find the max duration for the closed station and open station

In [80]:
%%bigquery --project smart-tracer-338719
SELECT start_station_name, status, duration
FROM (SELECT start_station_name, status, duration_minutes as duration
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`as station JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as trips ON station.name=trips.start_station_name
GROUP BY start_station_name, duration_minutes, status)
WHERE status='closed'
ORDER BY duration DESC
LIMIT 1


,start_station_name,status,duration
0,23rd & Rio Grande,closed,6643


Now I will find the max duration for the active station 

In [67]:
%%bigquery --project smart-tracer-338719
SELECT start_station_name, status, duration
FROM (SELECT start_station_name, status, duration_minutes as duration
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`as station JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as trips ON station.name=trips.start_station_name
GROUP BY start_station_name, duration_minutes, status)
WHERE status='active'
ORDER BY duration DESC
LIMIT 1

,start_station_name,status,duration
0,Barton Springs @ Kinney Ave,active,21296


Now I count average duration grouped by status and count total number of open and closed station

In [73]:
%%bigquery --project smart-tracer-338719
SELECT status, AVG(duration) as avg_duration, COUNT(status) as total
FROM (SELECT start_station_name, status, duration_minutes as duration
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`as station JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as trips ON station.name=trips.start_station_name
GROUP BY start_station_name, duration_minutes, status)
GROUP BY status

,status,avg_duration,total
0,closed,219.548847,4207
1,active,346.568084,22252
